# PDF

## Using PyPDF

In [ ]:
!pip install --upgrade --quiet langchain langchain-openai langchain-community pypdf faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = 'sk-******************************************'

`PyPDFLoader`를 사용하여 PDF파일을 로드합니다.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/layout-parser-readthedocs-io-en-latest.pdf")
pages = loader.load_and_split()

In [ ]:
pages[0].page_content

'Layout Parser\nRelease 0.3.2\nLayout Parser Contributors\nApr 02, 2022'

In [ ]:
pages[0].metadata

{'source': '/content/layout-parser-readthedocs-io-en-latest.pdf', 'page': 0}

`FAISS`에 문서를 임베딩 합니다.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("How will the community be engaged?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

7: Layout Parser, Release 0.3.2
4 Chapter 2. Model Zoo
53: Layout Parser, Release 0.3.2
50 Chapter 8. Text Recognition Tool


### PDF Usage

`LLMChain`을 사용하여 문서를 요약하는 Chain을 생성합니다.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

map_template = """다음은 문서 중 일부 내용입니다
{pages}
이 문서 목록을 기반으로 주요 내용을 요약해 주세요.
답변:"""

map_prompt = PromptTemplate.from_template(map_template)

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-16k')
map_chain = LLMChain(llm=llm, prompt=map_prompt)

map_chain.run(docs)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'주어진 문서 목록에는 "Layout Parser"라는 소프트웨어의 버전 0.3.2에 대한 내용이 포함되어 있습니다. 첫 번째 문서는 "Chapter 2. Model Zoo"에 대한 내용이며, 두 번째 문서는 "Chapter 8. Text Recognition Tool"에 대한 내용입니다.'

In [ ]:
reduce_template = """다음은 요약의 집합입니다:
{doc_summaries}
이것들을 바탕으로 통합된 요약을 만들어 주세요.
답변:"""

reduce_prompt = PromptTemplate.from_template(reduce_template)

reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

reduce_chain.run(docs)

'Layout Parser의 최신 버전은 0.3.2이며, Model Zoo와 Text Recognition Tool이 포함되어 있습니다. Model Zoo는 2장에서, Text Recognition Tool은 8장에서 소개되었습니다.'

`StuffDocumentsChain`은 문서 목록을 가져와서 먼저 단일 문자열로 결합합니다.

`document_variable_name`으로 설정된 변수 이름을 사용하여 입력에 해당 새 문자열을 추가합니다.

해당 입력은 `llm_chain` 으로 전달됩니다.

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain

combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="doc_summaries"
)

combine_documents_chain.run(docs)

'Layout Parser의 Release 0.3.2에는 Model Zoo와 Text Recognition Tool이 포함되어 있습니다.'

`ReduceDocumentsChain문서`를 반복적으로 줄여 문서를 결합합니다.

`combine_documents_chain`: 이전 결과를 모두 이 체인에 전달하고 이 체인의 출력이 최종 결과로 반환됩니다.

`collapse_documents_chain`: 전달된 문서가 너무 많아서 한 번에 `Combine_documents_chain` 에 모두 전달할 수 없는 경우에 사용됩니다.

In [ ]:
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,
    token_max=4000,
)

reduce_documents_chain.run(docs)

'Layout Parser의 Release 0.3.2에는 Model Zoo와 Text Recognition Tool이 포함되어 있습니다.'

`MapReduceDocumentsChain`은 먼저 LLM을 통해 각 문서를 전달한 다음 ReduceDocumentsChain을 사용하여 문서를 줄입니다.

먼저 각 문서에서 개별적으로 `llm_chain을` 호출하여 `page_content` 및 기타 `kwargs`를 전달합니다.

In [ ]:
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="pages",
    return_intermediate_steps=False,
)

map_reduce_chain.run(docs)

'이 문서는 "Layout Parser"의 0.3.2 버전에 대한 내용을 다루고 있습니다. 문서는 총 4장으로 구성되어 있으며, 그 중 2장은 "Model Zoo"에 대한 내용을 다루고 있습니다. "Model Zoo"에는 무엇인가에 대한 구체적인 설명이 포함되어 있을 것으로 예상됩니다. 또한, 문서의 50장은 "Text Recognition Tool"에 대한 내용을 다루고 있습니다.'

## Using PyPDFium2

`PyPDFium2Loader`는 pypdfium2 및 문자 수준의 청크를 사용하여 PDF를 로드합니다.

In [ ]:
from langchain_community.document_loaders import PyPDFium2Loader

In [ ]:
!pip install --quiet pypdfium2

In [ ]:
loader = PyPDFium2Loader("/content/layout-parser-readthedocs-io-en-latest.pdf")

In [ ]:
data = loader.load()

/usr/local/lib/python3.10/dist-packages/pypdfium2/_helpers/textpage.py:81: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [ ]:
pages = loader.load_and_split()

In [ ]:
pages[0].page_content

'Layout Parser\r\nRelease 0.3.2\r\nLayout Parser Contributors\r\nApr 02, 2022'

In [ ]:
pages[0].metadata

{'source': '/content/layout-parser-readthedocs-io-en-latest.pdf', 'page': 0}

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("How will the community be engaged?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

7: Layout Parser, Release 0.3.2
4 Chapter 2. Model Zoo
53: Layout Parser, Release 0.3.2
50 Chapter 8. Text Recognition Tool


## Using PyMuPDF

`PyMuPDFLoader`는 PDF 구문 분석 옵션 중 가장 빠르며 PDF 및 해당 페이지에 대한 자세한 메타데이터를 포함할 뿐만 아니라 페이지당 하나의 문서를 반환합니다.

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

In [ ]:
!pip install --upgrade --quiet pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 22.6 MB/s eta 0:00:00


In [ ]:
loader = PyMuPDFLoader("/content/layout-parser-readthedocs-io-en-latest.pdf")

In [ ]:
pages = loader.load_and_split()

In [ ]:
pages[0].page_content

'Layout Parser\nRelease 0.3.2\nLayout Parser Contributors\nApr 02, 2022'

In [ ]:
pages[0].metadata

{'source': '/content/layout-parser-readthedocs-io-en-latest.pdf',
 'file_path': '/content/layout-parser-readthedocs-io-en-latest.pdf',
 'page': 0,
 'total_pages': 69,
 'format': 'PDF 1.5',
 'title': 'Layout Parser',
 'author': 'Layout Parser Contributors',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref package',
 'producer': 'pdfTeX-1.40.18',
 'creationDate': 'D:20220402174803Z',
 'modDate': 'D:20220402174803Z',
 'trapped': ''}

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("How will the community be engaged?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

7: Layout Parser, Release 0.3.2
4
Chapter 2. Model Zoo
53: Layout Parser, Release 0.3.2
50
Chapter 8. Text Recognition Tool


## PyPDF Directory

`PyPDFDirectoryLoader`는 디렉토리에서 PDF파일을 로드합니다.

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [ ]:
loader = PyPDFDirectoryLoader("/content/PyPdf")

In [ ]:
docs = loader.load()

In [ ]:
len(docs)

104

In [ ]:
docs[0].page_content

'Layout Parser\nRelease 0.3.2\nLayout Parser Contributors\nApr 02, 2022'

In [ ]:
docs[0].metadata

{'source': '/content/PyPdf/layout-parser-readthedocs-io-en-latest.pdf',
 'page': 0}

## **Using PDFPlumber**

`PDFPlumberLoader`는 `PyMuPDF`와 마찬가지로 출력 문서에는 PDF와 해당 페이지에 대한 자세한 메타데이터가 포함되어 있으며 페이지당 하나의 문서를 반환합니다.

In [ ]:
!pip install --upgrade --quiet pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.0 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader

In [ ]:
loader = PDFPlumberLoader("/content/PyPdf/2403.09876.pdf")

In [ ]:
data = loader.load()

In [ ]:
data[0].page_content

'WHICH SHAPES CAN APPEAR IN A CURVE SHORTENING\nFLOW SINGULARITY?\nSIGURDB.ANGENENT,EVANPATRICKDAVIS,ELLIEDECLEENE,\nPAIGEELLINGSON,ZIHENGFENG,EDGARGEVORGYAN,ARISLEMMENES,\nALEXMOON,TYLERJOSEPHTOMMASI,ANDYAMINZHOU\nAbstract. Westudypossibletanglesthatcanoccurinsingularitiesofsolu-\ntions to plane Curve Shortening Flow. We exhibit solutions in which more\ncomplicated tangles with more than one self-intersection disappear into a\nsingular point. It seems that there are many examples of this kind and\nthat a complete classification presents a problem similar to the problem of\nclassifying all knots in R3. As a particular example, we introduce the so-\ncalled n-loop curves, which generalize Matt Grayson’s Figure-Eight curve,\nandweconjectureageneralizationoftheCoiculescu-Schwarzasymptoticbow-\ntie result, namely, a vanishing n-loop, when rescaled anisotropically to fit\na square bounding box, converges to a “squeezed bow-tie,” i.e. the curve\n{(x,y) : |x| ≤ 1,y = ±xn−1}∪{(±1,y) : |y| ≤ 1}.

In [ ]:
data[0].metadata

{'source': '/content/PyPdf/2403.09876.pdf',
 'file_path': '/content/PyPdf/2403.09876.pdf',
 'page': 0,
 'total_pages': 21,
 'Author': '',
 'CreationDate': 'D:20240318010436Z',
 'Creator': 'LaTeX with hyperref',
 'Keywords': '',
 'ModDate': 'D:20240318010436Z',
 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'Producer': 'pdfTeX-1.40.25',
 'Subject': '',
 'Title': '',
 'Trapped': 'False'}

### **glob을 통해서 폴더 단위로 PDFPlumberLoader 사용하기**

In [ ]:
import glob
from langchain.document_loaders import PDFPlumberLoader

pdf_files = glob.glob("/content/PyPdf/*.pdf")

docs = []
for pdf_file in pdf_files:
    loader = PDFPlumberLoader(pdf_file)
    docs.extend(loader.load())

In [ ]:
len(docs)

104

In [ ]:
docs[0].page_content

'Layout Parser\nRelease 0.3.2\nLayout Parser Contributors\nApr 02, 2022\n'

In [ ]:
docs[0].metadata

{'source': '/content/PyPdf/layout-parser-readthedocs-io-en-latest.pdf',
 'file_path': '/content/PyPdf/layout-parser-readthedocs-io-en-latest.pdf',
 'page': 0,
 'total_pages': 69,
 'Author': 'Layout Parser Contributors',
 'Title': 'Layout Parser',
 'Subject': '',
 'Creator': 'LaTeX with hyperref package',
 'Producer': 'pdfTeX-1.40.18',
 'Keywords': '',
 'CreationDate': 'D:20220402174803Z',
 'ModDate': 'D:20220402174803Z',
 'Trapped': 'False',
 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) kpathsea version 6.2.3'}